In [1]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
from datetime import date
import pandas as pd

today=date.today()

In [2]:
data_dir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02-S/'
gridfile='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/mesh_mask_eNATL60_3.6_lev1.nc4'
dsgrid=xr.open_dataset(gridfile,chunks={'x':1000,'y':1000})

In [3]:
boxes_LS=pd.read_csv('../make_boxes/boxes_LS_1x1_eNATL60.csv',sep = '\t',index_col=0)
imin_LS=boxes_LS['imin']
imax_LS=boxes_LS['imax']
jmin_LS=boxes_LS['jmin']
jmax_LS=boxes_LS['jmax']
box_name=boxes_LS.index

In [4]:
def dx_var(data,e1):
    dx_var = (data.shift(x=-1) - data)/e1
    return dx_var
def dy_var(data,e2):
    dy_var = (data.shift(y=-1) - data)/e2
    return dy_var
    

In [15]:
filetyps = {'votemper' : 'gridT', 'vosaline' : 'gridS','vozocrtx' : 'gridU', 'vomecrty' : 'gridV','vovecrtz' : 'gridW'}
filedeps = {'votemper' : 'deptht','vosaline' : 'deptht','vozocrtx' : 'depthu', 'vomecrty' : 'depthv','vovecrtz':'depthw'}
filee1 = {'votemper' : 'e1t','vosaline' : 'e1t','vozocrtx' : 'e1u', 'vomecrty' : 'e1v','vovecrtz':'e1f'}
filee2 = {'votemper' : 'e2t','vosaline' : 'e2t','vozocrtx' : 'e2u', 'vomecrty' : 'e2v','vovecrtz':'e2f'}


def compute_all_profiles_temperature(date,ibox):
    var='votemper'
    filename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_'+filetyps[var]+'_'+date+'-'+date+'.nc'))
    file=filename[0]
    ds=xr.open_dataset(file)
    data_box=ds[str(var)][:,:,jmin_LS[ibox]-10:jmax_LS[ibox]+10,imin_LS[ibox]-10:imax_LS[ibox]+10]
    e1=dsgrid[str(filee1[var])][0,jmin_LS[ibox]-10:jmax_LS[ibox]+10,imin_LS[ibox]-10:imax_LS[ibox]+10]
    e2=dsgrid[str(filee2[var])][0,jmin_LS[ibox]-10:jmax_LS[ibox]+10,imin_LS[ibox]-10:imax_LS[ibox]+10]
    data_dx=dx_var(data_box,e1)
    data_dy=dy_var(data_box,e2)
    profile_data=data_box[:,:,10:-10,10:-10].mean(dim={'x','y','time_counter'})
    profile_data_dx=data_dx[:,:,10:-10,10:-10].mean(dim={'x','y','time_counter'})
    profile_data_dy=data_dy[:,:,10:-10,10:-10].mean(dim={'x','y','time_counter'})
    return profile_data,profile_data_dx,profile_data_dy



In [16]:
%time profile_data,profile_data_dx,profile_data_dy=compute_all_profiles_temperature('20090701',0)

CPU times: user 3min 57s, sys: 13.7 s, total: 4min 10s
Wall time: 6min 40s
